# GIL (Global Interpeter Lock) в Python

Начнем сразу с примера программы:

In [11]:
def count(n):
    while n > 0:
        n -= 1
x = 100000000

Запустим два раза и измерим время:

In [12]:
%%time
count(x)
count(x)

CPU times: user 8.95 s, sys: 28 ms, total: 8.98 s
Wall time: 9.07 s


Также запустим две функции, но теперь они будут выполняться параллельно:

In [13]:
from threading import Thread

In [14]:
%%time
t1 = Thread(target=count,args=(x,))
t1.start()
t2 = Thread(target=count,args=(x,))
t2.start()
t1.join()
t2.join()

CPU times: user 9.33 s, sys: 72.2 ms, total: 9.41 s
Wall time: 9.4 s


В любой момент может выполняться только один поток Python. Глобальная блокировка интерпретатора — GIL — тщательно контролирует выполнение тредов. GIL гарантирует каждому потоку эксклюзивный доступ к переменным интерпретатора (и соответствующие вызовы C-расширений работают правильно).

Потоки удерживают GIL, пока выполняются. Однако они освобождают его при блокировании для операций ввода-вывода. Каждый раз, когда поток вынужден ждать, другие, готовые к выполнению, потоки используют свой шанс запуститься.

При работе с CPU-зависимыми потоками, которые никогда не производят операции ввода-вывода, интерпретатор периодически проводит проверку («the periodic check»).

По умолчанию это происходит каждые 100 «тиков»(тики неделимые инструкции python), но этот параметр можно изменить с помощью sys.setcheckinterval(). Интервал проверки — глобальный счетчик, абсолютно независимый от порядка переключения потоков. (В новых версиях python, проверка происходит по времени, а не по тикам).

При периодической проверке в главном потоке запускаются обработчики сигналов, если таковые имеются. Затем GIL отключается и включается вновь. На этом этапе обеспечивается возможность переключения нескольких CPU-зависимых потоков (при кратком освобождении GIL другие треды имеют шанс на запуск).

### Вопрос 1. А зачем нужен GIL? 

Python подсчитывает количество ссылок для корректного управления памятью. Это означает, что созданные в Python объекты имеют переменную подсчёта ссылок, в которой хранится количество всех ссылок на этот объект. Как только эта переменная становится равной нулю, память, выделенная под этот объект, освобождается.

Вот небольшой пример кода, демонстрирующий работу переменных подсчёта ссылок:

In [16]:
import sys
a = []
b = a
sys.getrefcount(a)

3

В этом примере количество ссылок на пустой массив равно 3. На этот массив ссылаются: переменная a, переменная b и аргумент, переданный функции sys.getrefcount().

Проблема, которую решает GIL, связана с тем, что в многопоточном приложении сразу несколько потоков могут увеличивать или уменьшать значения этого счётчика ссылок. Это может привести к тому, что память очистится неправильно и удалится тот объект, на который ещё существует ссылка.

Счётчик ссылок можно защитить, добавив блокираторы на все структуры данных, которые распространяются по нескольким потокам. В таком случае счётчик будет изменяться исключительно последовательно.

Но добавление блокировки к нескольким объектам может привести к появлению другой проблемы — взаимоблокировки (англ. deadlocks), которая получается только если блокировка есть более чем на одном объекте. К тому же эта проблема тоже снижала бы производительность из-за многократной установки блокираторов.

### Вопрос 2. До этого мы говорили про потоки. А чем поток отличается от процесса?

### Задание 1. Попробуйте решить задачу выше с помощью библиотеки PARL. 

https://parl.readthedocs.io/en/latest/parallel_training/setup.html

### Дополнительные материалы:
https://habr.com/ru/post/84629/

https://github.com/PaddlePaddle/PARL

https://tproger.ru/translations/global-interpreter-lock-guide/